Initial set up, including search terms.

In [1]:
import sys

In [2]:
import wosis
import pandas as pd
import metaknowledge as mk
import json

C:\Users\douglasd\AppData\Local\Continuum\miniconda3\envs\biblio\lib\site-packages\fuzzywuzzy-0.17.0-py3.6.egg\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
wos_config = wosis.load_config("config.yml")

In [4]:
search_terms={
    "inclusive_kw":("sensitivity analysis", "uncertainty analysis", "uncertainty quantification", "uncertainty propagation", 
                    "local sensitivity analysis", "LSA", "one-at-a-time", 'exploratory modeling',
                    "OAT", "global sensitivity analysis", "GSA", "all-at-a-time", "AAT"),
"subject_area":("ENVIRONMENTAL SCIENCES", "WATER RESOURCES", "ENGINEERING ENVIRONMENTAL", "INTERDISCIPLINARY APPLICATIONS"),
}

In [5]:
topics = [wosis.build_query(search_terms),]
topics

['TS=("sensitivity analysis" OR "uncertainty analysis" OR "uncertainty quantification" OR "uncertainty propagation" OR "local sensitivity analysis" OR "LSA" OR "one-at-a-time" OR "exploratory modeling" OR "OAT" OR "global sensitivity analysis" OR "GSA" OR "all-at-a-time" OR "AAT") AND WC=("ENVIRONMENTAL SCIENCES" OR "WATER RESOURCES" OR "ENGINEERING ENVIRONMENTAL" OR "INTERDISCIPLINARY APPLICATIONS")']

In [ ]:
%%time
overwrite = False
id_to_query, query_recs = wosis.query(topics, overwrite, wos_config, time_span={'begin':'2000-01-01','end':'2017-12-31'})

Authenticated (SID: 5COml39ikZnPN1oweh7)


c:\userdata\douglasd\project\biblio\wosis\wosis\query.py:304: UserWarning: Getting referenced works...
  warnings.warn("Getting referenced works...")


c:\userdata\douglasd\project\biblio\wosis\wosis\query.py:357: UserWarning: A reference had more than 100 citations. This can take a long time and the cache file can get quite large...
  "A reference had more than {} citations. This can take a long time and the cache file can get quite large...".format(batch_size))


Server Error Msg: b"Server raised fault: '(ISE0002) Exception occurred processing request. Reason: Error received from a server supporting the Web service. Cause: Supporting server error message = [ID limit 2500 exceeded]. Supporting server error code = [Server.IDLimit]. Remedy: If this is a problem with your request, correct your request and submit it again. Otherwise, call customer support. '"
Authenticated (SID: 6EqtihVwPiR8A6OC2ph)
Server Error Msg: b"Server raised fault: '(ISE0002) Exception occurred processing request. Reason: Error received from a server supporting the Web service. Cause: Supporting server error message = [ID limit 2500 exceeded]. Supporting server error code = [Server.IDLimit]. Remedy: If this is a problem with your request, correct your request and submit it again. Otherwise, call customer support. '"
Authenticated (SID: 5BzUXx29f8zu1UFBBj2)
Server Error Msg: b"Server raised fault: '(ISE0002) Exception occurred processing request. Reason: Error received from a

In [7]:
with open('tmp/example_query_id.txt', 'w') as file:
    file.write(json.dumps(id_to_query, indent=2))

In [8]:
query_id=list(id_to_query)[0]
print("Query ID: ", query_id, "\nQuery String:", id_to_query[query_id], "\n")
RC=mk.RecordCollection("tmp/{}.txt".format(query_id))
print("Corpora consists of", len(RC), "Publications")

Query ID:  756d39801152fe5f5f4ad3a3df9b6a30_2010-01-01-2017-12-31 
Query String: TS=("sensitivity analysis" OR "uncertainty analysis" OR "uncertainty quantification" OR "uncertainty propagation" OR "local sensitivity analysis" OR "LSA" OR "one-at-a-time" OR "OAT" OR "global sensitivity analysis" OR "GSA" OR "all-at-a-time" OR "AAT") AND WC=("ENVIRONMENTAL SCIENCES" OR "WATER RESOURCES" OR "ENGINEERING ENVIRONMENTAL" OR "INTERDISCIPLINARY APPLICATIONS") 

Corpora consists of 15754 Publications


In [9]:
RC = list(query_recs.values())[0]
print("Corpora consists of", len(RC), "Publications")

Corpora consists of 15754 Publications


In [10]:
from datetime import datetime
repset_df = pd.DataFrame(RC.forNLP(extraColumns=["AU", "SO", "DE"]))
repset_df = repset_df.drop(["keywords", "abstract"], axis=1)

assert len(repset_df.id.unique()) == len(repset_df.id), "Duplicate records found!"

repset_fn = "../data/{}.csv".format(query_id)
with open(repset_fn, 'w') as fn:
    fn.write("# Data from Clarivate Analytics' Web of Science, retrieved 5 December 2018\n")
    fn.write("# This file generated on {}\n".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    repset_df.index.name = "item"
    repset_df.to_csv(fn)

In [11]:
# Helper functions for plotting
import wosis.analysis.plotting as wos_plot

# Analysis and search functions
from wosis.analysis import (preview_matches, search_records, keyword_matches, 
                            find_pubs_by_authors, preview_matches_by_keyword, get_unique_kw_titles,
                            remove_by_journals, remove_by_title)

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# This just helps make the plots look nicer
sns.set_style('darkgrid')
sns.set_context('paper', font_scale=2.0)